In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Loading the Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,precision_score,recall_score
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from joblib import Parallel, delayed, parallel_backend
from tensorflow.keras.backend import clear_session

# Loading the Data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Major Project/creditcard.csv")
df.head(5)

# Scaling the Amount and Time

In [ ]:
df[['Scaled_Time', 'Scaled_Amount']] = StandardScaler().fit_transform(df[['Time', 'Amount']])
df = df.drop(['Time', 'Amount'], axis=1)

In [ ]:
X = df.drop(['Class'], axis=1)
y = df['Class']

# Splitting the Dataset into Train and Test Set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

# Creation of CNN

In [ ]:
def create_cnn():
    model = keras.Sequential([
        layers.Input(shape=(X_train.shape[1], 1)),
        layers.Conv1D(filters=16, kernel_size=3, activation='relu', padding='same'),
        layers.MaxPooling1D(pool_size=2),
        layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
        layers.MaxPooling1D(pool_size=2),
        layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        layers.GlobalMaxPooling1D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=keras.optimizers.AdamW(learning_rate=0.001,weight_decay=1e-5),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Training the CNN Model

In [ ]:
# Create the CNN model
cnn_model = create_cnn()

# Callbacks: Early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Train the CNN model
history = cnn_model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping, lr_scheduler],
    verbose=1
)

# Predicting and Evaluating the CNN Model

In [ ]:
y_predicted = (cnn_model.predict(X_test) > 0.5).astype(int)
report=classification_report(y_test, y_predicted)
print('Classification report:\n', classification_report(y_test, y_predicted))
cm = confusion_matrix(y_true=y_test, y_pred=y_predicted)
print('Confusion matrix:\n', cm)
accuracy=accuracy_score(y_test, y_predicted)
print('Accuracy:', accuracy_score(y_test, y_predicted))
precision = precision_score(y_test, y_predicted, pos_label=1.0)
print('precision : ',precision)
recall = recall_score(y_test, y_predicted, pos_label=1.0)
print('recall rate : ',recall)

# Saving the Model

In [ ]:
import os
import seaborn as sns
# Output Directory
output_path = "/content/drive/MyDrive/Major Project/Model_Outputs"
os.makedirs(output_path, exist_ok=True)

# Save model
model_save_path = os.path.join(output_path, "cnn_ns_model.keras")
cnn_model.save(model_save_path)
print(f"Model saved at: {model_save_path}")

# Save classification report and confusion matrix
metrics_path = os.path.join(output_path, "cnn_ns_model_metrics.txt")
with open(metrics_path, "w") as f:
    f.write("Classification Report:\n")
    f.write(report)
    f.write("\nConfusion Matrix:\n")
    f.write(str(cm))
    f.write(f"\nAccuracy: {accuracy:.4f}")
    f.write(f"\nPrecision: {precision:.4f}")
    f.write(f"\nRecall: {recall:.4f}")
print(f"Metrics saved at: {metrics_path}")

# Save confusion matrix plot
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
conf_matrix_path = os.path.join(output_path, "cnn_ns_confusion_matrix.png")
plt.savefig(conf_matrix_path)
plt.close()
print(f"Confusion matrix plot saved at: {conf_matrix_path}")

# Loading the Saved Model

In [ ]:
import joblib
from joblib import load
import os
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,precision_score,recall_score, roc_auc_score,roc_curve,precision_recall_curve,auc,roc_curve,f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model

# Path to load the saved model
model_load_path = "/content/drive/MyDrive/Major Project/Model_Outputs/cnn_ns_model.keras"

# Load the CNN model
cnn_model_loaded = load_model(model_load_path)

# Predict on test data
y_pred_prob = cnn_model_loaded.predict(X_test)
y_pred_test = (y_pred_prob > 0.5).astype(int)

# Metrics: Precision, Recall, F1-Score
precision = precision_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
accuracy = accuracy_score(y_test, y_pred_test)

print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-Score: {f1:.3f}")

# Classification Report and Confusion Matrix
report = classification_report(y_test, y_pred_test)
print("\nClassification Report:\n", report)

cm = confusion_matrix(y_test, y_pred_test)
print("\nConfusion Matrix:\n", cm)

# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# AUC-ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color="darkorange", label=f"ROC Curve (AUC = {roc_auc:.3f})")
plt.title("AUC-ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.grid()
plt.show()

# AUC-PRC Curve
precision_pr, recall_pr, _ = precision_recall_curve(y_test, y_pred_prob)
pr_auc = auc(recall_pr, precision_pr)

plt.figure(figsize=(8, 6))
plt.plot(recall_pr, precision_pr, color="purple", lw=2, label=f"PR Curve (AUC = {pr_auc:.3f})")
plt.title("AUC-PRC Curve")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="lower left")
plt.grid()
plt.show()